# Neighborhood suggestion for vegetarian/vegan restaurant in Hamburg (Germany)

## Import all necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import re
print('Libraries imported.')

import requests

from bs4 import BeautifulSoup

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

## Import data table with Hamburg districts and quarters from wikipedia using beautiful Soup

In [2]:
page = requests.get('https://de.wikipedia.org/wiki/Liste_der_Bezirke_und_Stadtteile_Hamburgs').text

In [3]:
soup = BeautifulSoup(page,'lxml')

## Find second table of wikipeda page and get all rows and columns

In [4]:
My_table = soup.findAll("table", class_='wikitable')[1]
rows = My_table.find_all('tr')
columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]

## Append all values to a pandas data frame

In [5]:
df = pd.DataFrame(columns=columns)
for i in range(1, len(rows)):
               tds = rows[i].find_all('td')
# Clean and append all values to the pandas DataFrame
               values = [td.text.replace('\n','') for td in tds]         
               df = df.append(pd.Series(values, index=columns), ignore_index=True)
df.head()

,Stadtteil,Ortsteile,Bezirk,Fläche(km²),Einwohner,Bevölkerungsdichte(Einwohner/km²),Koordinaten,Karte
0,Hamburg-Altstadt!Hamburg-Altstadt,101!101–102,Hamburg-Mitte,"02,4!2,4",2272,947,"53° 33′ 0″ N, 10° 0′ 0″ O",
1,HafenCity!HafenCity,103!103–104,Hamburg-Mitte,"02,2!2,2",4592,2087,"53° 32′ 28″ N, 10° 0′ 1″ O",
2,Neustadt!Neustadt,105!105–108,Hamburg-Mitte,"02,3!2,3",12.920,5617,"53° 33′ 7″ N, 9° 59′ 8″ O",
3,St.Pauli!St. Pauli,109!109–112,Hamburg-Mitte,"02,5!2,5",22.436,8974,"53° 33′ 25″ N, 9° 57′ 50″ O",
4,St.Georg!St. Georg,113!113–114,Hamburg-Mitte,"02,4!2,4",11.384,4743,"53° 33′ 18″ N, 10° 0′ 44″ O",


## Clean up and customize the data frame, translate headers from German to English

In [6]:
df['District'] = df['Stadtteil'].str.split('!', 1).str[0]
df['District_No'] = df['Ortsteile'].str.split('!', 1).str[1]
df['Area_sqkm'] = df['Fläche(km²)'].str.split('!', 1).str[1]
df[['Latitude', 'Longitude']] = df.Koordinaten.str.split(',',1, expand=True); df
df['Longitude'] = df['Longitude'].str.replace('O', 'E', regex=False)
df['Latitude'] = df['Latitude'].str.replace(' ','')
df['Longitude'] = df['Longitude'].str.replace(' ','')
df = df.drop(columns=['Stadtteil','Ortsteile','Fläche(km²)','Karte'])
df = df.rename(columns={"Bezirk": "Borough", "Einwohner": "Inhabitants", "Bevölkerungsdichte(Einwohner/km²)": "Population_density_per_sqkm"})

In [7]:
df = df[['Borough','District_No', 'District', 'Inhabitants', 'Area_sqkm', 'Population_density_per_sqkm', 'Latitude', 'Longitude']]
df.head()

,Borough,District_No,District,Inhabitants,Area_sqkm,Population_density_per_sqkm,Latitude,Longitude
0,Hamburg-Mitte,101–102,Hamburg-Altstadt,2272,"2,4",947,53° 33′ 0″ N,10° 0′ 0″ E
1,Hamburg-Mitte,103–104,HafenCity,4592,"2,2",2087,53° 32′ 28″ N,10° 0′ 1″ E
2,Hamburg-Mitte,105–108,Neustadt,12.920,"2,3",5617,53° 33′ 7″ N,9° 59′ 8″ E
3,Hamburg-Mitte,109–112,St.Pauli,22.436,"2,5",8974,53° 33′ 25″ N,9° 57′ 50″ E
4,Hamburg-Mitte,113–114,St.Georg,11.384,"2,4",4743,53° 33′ 18″ N,10° 0′ 44″ E


In [8]:
# get the geographical coordinates of Hamburg (Germany)
address = 'Hamburg, HH'

geolocator = Nominatim(user_agent="hh_explorer")
location = geolocator.geocode(address)
HHlatitude = location.latitude
HHlongitude = location.longitude
print('The geographical coordinate of Hamburg are {}, {}.'.format(HHlatitude, HHlongitude))

The geographical coordinate of Hamburg are 53.550341, 10.000654.


In [9]:
# check columns 'Longitude' and 'Latitude'
df.Longitude.head(),df.Latitude.head()

(0     10° 0′ 0″ E
 1     10° 0′ 1″ E
 2     9° 59′ 8″ E
 3    9° 57′ 50″ E
 4    10° 0′ 44″ E
 Name: Longitude, dtype: object, 0     53° 33′ 0″ N
 1    53° 32′ 28″ N
 2     53° 33′ 7″ N
 3    53° 33′ 25″ N
 4    53° 33′ 18″ N
 Name: Latitude, dtype: object)

In [10]:
# functions to convert longitude and latitude values to decimal coordinates

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'W' or direction == 'S':
        dd *= -1
    return dd;

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

def parse_dms(dms):
    parts = re.split('[^\d\w]+', dms)
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])

    return (lat)

In [11]:
# add all decimal latitude and longitude values to lists
n = len(df)
LatList = []
LongList = []

for i in range(n):
  Lat_d = round(parse_dms(df.Latitude[i].replace(" ", "")),4)
  Long_d = round(parse_dms(df.Longitude[i].replace(" ", "")),4)
  LatList.append(Lat_d)   
  LongList.append(Long_d)
    
# add the lists with decimal values to the existing data frame
df['Lat'] = LatList
df['Long'] = LongList

df.head()

,Borough,District_No,District,Inhabitants,Area_sqkm,Population_density_per_sqkm,Latitude,Longitude,Lat,Long
0,Hamburg-Mitte,101–102,Hamburg-Altstadt,2272,"2,4",947,53° 33′ 0″ N,10° 0′ 0″ E,53.5500,10.0000
1,Hamburg-Mitte,103–104,HafenCity,4592,"2,2",2087,53° 32′ 28″ N,10° 0′ 1″ E,53.5411,10.0003
2,Hamburg-Mitte,105–108,Neustadt,12.920,"2,3",5617,53° 33′ 7″ N,9° 59′ 8″ E,53.5519,9.9856
3,Hamburg-Mitte,109–112,St.Pauli,22.436,"2,5",8974,53° 33′ 25″ N,9° 57′ 50″ E,53.5569,9.9639
4,Hamburg-Mitte,113–114,St.Georg,11.384,"2,4",4743,53° 33′ 18″ N,10° 0′ 44″ E,53.5550,10.0122


## Create Hamburg map and add markers for districts to get a first overview

In [12]:
# create map of Hamburg using latitude and longitude values
map_hamburg = folium.Map(location=[HHlatitude, HHlongitude], zoom_start=11)

# add markers to Hamburg map
for Lat, Long, Borough, District in zip(df['Lat'], df['Long'], df['Borough'], df['District']):
    label = '{}, {}'.format(District, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        [Lat, Long],
        popup=label).add_to(map_hamburg)
   
map_hamburg

## Generate a subset of the data frame

In [13]:
# convert population density to int
df.Population_density_per_sqkm = df.Population_density_per_sqkm.astype(int)
# generate a data frame which contains central boroughs only and a population density >4000 per km²
df_central_distr = df[df.Borough.isin(['Hamburg-Mitte','Altona']) & (df.Population_density_per_sqkm >= 4000)].reset_index(drop=True)

In [14]:
df_central_distr

,Borough,District_No,District,Inhabitants,Area_sqkm,Population_density_per_sqkm,Latitude,Longitude,Lat,Long
0,Hamburg-Mitte,105–108,Neustadt,12.920,"2,3",5617,53° 33′ 7″ N,9° 59′ 8″ E,53.5519,9.9856
1,Hamburg-Mitte,109–112,St.Pauli,22.436,"2,5",8974,53° 33′ 25″ N,9° 57′ 50″ E,53.5569,9.9639
2,Hamburg-Mitte,113–114,St.Georg,11.384,"2,4",4743,53° 33′ 18″ N,10° 0′ 44″ E,53.5550,10.0122
3,Hamburg-Mitte,119–120,Borgfelde,7696,"0,8",9620,53° 33′ 17″ N,10° 2′ 4″ E,53.5547,10.0344
4,Hamburg-Mitte,121–127,Hamm,38.773,"4,0",9693,53° 33′ 39″ N,10° 3′ 28″ E,53.5608,10.0578
5,Hamburg-Mitte,128–129,Horn,38.799,"5,9",6576,53° 33′ 14″ N,10° 5′ 24″ E,53.5539,10.0900
6,Hamburg-Mitte,130,Billstedt,70.355,"17,0",4139,53° 32′ 26″ N,10° 6′ 4″ E,53.5406,10.1011
7,Altona,201–206,Altona-Altstadt,29.305,"2,8",10466,53° 32′ 56″ N,9° 56′ 52″ E,53.5489,9.9478
8,Altona,207,Sternschanze,8095,"0,5",16190,53° 33′ 42″ N,9° 57′ 44″ E,53.5617,9.9622
9,Altona,208–210,Altona-Nord,24.153,"2,2",10979,53° 33′ 47″ N,9° 56′ 43″ E,53.5631,9.9453


In [15]:
# create a map of the district subset
map_hamburg = folium.Map(location=[HHlatitude, HHlongitude], zoom_start=12)

for Lat, Long, Borough, District in zip(df_central_distr['Lat'], df_central_distr['Long'], df_central_distr['Borough'], df_central_distr['District']):
    label = '{}, {}'.format(District, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        [Lat, Long],
        popup=label).add_to(map_hamburg)       

map_hamburg

## Use Foursquare to explore the neighborhoods of central Hamburg districts

In [16]:
CLIENT_ID = 'D3BGST0IOL3VDGWJD35EXZTISHC3Z42RUGX2UWOQ5NUCWXTN' # your Foursquare ID
CLIENT_SECRET = 'K1GLHABDR3IG5MSNTMZHPSZWXRXMQ55IXK0NUQH1SZ4HXPLM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D3BGST0IOL3VDGWJD35EXZTISHC3Z42RUGX2UWOQ5NUCWXTN
CLIENT_SECRET:K1GLHABDR3IG5MSNTMZHPSZWXRXMQ55IXK0NUQH1SZ4HXPLM


In [17]:
#df_central_distr.loc[0, 'District'],df_central_distr.loc[0, 'Lat'],df_central_distr.loc[0, 'Long']

('Neustadt', 53.5519, 9.9856)

In [20]:
# 150 venues that are in Neustadt within a radius of 1000 meters
LIMIT = 150
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df_central_distr.loc[0, 'Lat'], 
    df_central_distr.loc[0, 'Long'], 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=D3BGST0IOL3VDGWJD35EXZTISHC3Z42RUGX2UWOQ5NUCWXTN&client_secret=K1GLHABDR3IG5MSNTMZHPSZWXRXMQ55IXK0NUQH1SZ4HXPLM&v=20180605&ll=53.5519,9.9856&radius=1000&limit=150'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dc9d00dad178900393a1221'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Neustadt',
  'headerFullLocation': 'Neustadt, Hamburg',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 240,
  'suggestedBounds': {'ne': {'lat': 53.560900009000015,
    'lng': 10.00072083791351},
   'sw': {'lat': 53.54289999099999, 'lng': 9.97047916208649}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '52e12f23498eab3870ffa96c',
       'name': 'Public Coffee Roasters',
       'location': {'address': 'Wexstr. 28',
        'lat': 53.551372,
        'lng': 9.982597,
        'labeledLatLngs': [{'label': 'display',
          'lat': 53.551372,
          'ln

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Public Coffee Roasters,Coffee Shop,53.551372,9.982597
1,Tortue,Hotel,53.551437,9.986531
2,Die Bank,Restaurant,53.553501,9.987226
3,Zum Spätzle,Swabian Restaurant,53.551033,9.981608
4,Kardelen,Turkish Restaurant,53.550667,9.982271


In [26]:
#print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

In [24]:
#function to repeat the venues for all districts in central Hamburg
LIMIT = 500
#radius = 2000
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District_Latitude', 
                  'District_Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues)

In [25]:
central_venues = getNearbyVenues(names=df_central_distr['District'],
                                   latitudes=df_central_distr['Lat'],
                                   longitudes=df_central_distr['Long']
                                  )

Neustadt
St.Pauli
St.Georg
Borgfelde
Hamm
Horn
Billstedt
Altona-Altstadt
Sternschanze
Altona-Nord
Ottensen
Groß Flottbek
Lurup
Iserbrook


In [26]:
# size of the resulting dataframe
print(central_venues.shape)
central_venues

(1164, 7)


,District,District_Latitude,District_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Neustadt,53.5519,9.9856,Public Coffee Roasters,53.551372,9.982597,Coffee Shop
1,Neustadt,53.5519,9.9856,Tortue,53.551437,9.986531,Hotel
2,Neustadt,53.5519,9.9856,Zum Spätzle,53.551033,9.981608,Swabian Restaurant
3,Neustadt,53.5519,9.9856,Die Bank,53.553501,9.987226,Restaurant
4,Neustadt,53.5519,9.9856,Sautter + Lackmann,53.548779,9.985005,Bookstore
5,Neustadt,53.5519,9.9856,Kardelen,53.550667,9.982271,Turkish Restaurant
6,Neustadt,53.5519,9.9856,Bootshaus Kombüse,53.550701,9.982247,Restaurant
7,Neustadt,53.5519,9.9856,Nasch,53.555528,9.984067,Vegetarian / Vegan Restaurant
8,Neustadt,53.5519,9.9856,Erste Liebe,53.549359,9.985087,Café
9,Neustadt,53.5519,9.9856,Marinehof,53.549374,9.985253,German Restaurant


## Get the venues in central Hamburg with categories "Vegetarian/Vegan Restaurant" only

In [27]:
df_central_veg = central_venues[central_venues.Venue_Category.isin(['Vegetarian / Vegan Restaurant'])].reset_index(drop=True)
df_central_veg

,District,District_Latitude,District_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Neustadt,53.5519,9.9856,Nasch,53.555528,9.984067,Vegetarian / Vegan Restaurant
1,Neustadt,53.5519,9.9856,Happenpappen,53.557484,9.970221,Vegetarian / Vegan Restaurant
2,St.Pauli,53.5569,9.9639,Happenpappen,53.557484,9.970221,Vegetarian / Vegan Restaurant
3,St.Georg,53.5550,10.0122,Loving Hut,53.553915,10.019435,Vegetarian / Vegan Restaurant
4,St.Georg,53.5550,10.0122,Café Koppel,53.558304,10.011780,Vegetarian / Vegan Restaurant
5,Borgfelde,53.5547,10.0344,Loving Hut,53.553915,10.019435,Vegetarian / Vegan Restaurant
6,Borgfelde,53.5547,10.0344,Café Koppel,53.558304,10.011780,Vegetarian / Vegan Restaurant
7,Sternschanze,53.5617,9.9622,Happenpappen,53.557484,9.970221,Vegetarian / Vegan Restaurant
8,Ottensen,53.5539,9.9178,Froindlichst,53.556734,9.921648,Vegetarian / Vegan Restaurant


In [28]:
#delete double entries: venues 'Happenpappen' and 'Café Koppel'
df_central_vegetarian = df_central_veg.drop([1,6,7]).reset_index(drop=True)
df_central_vegetarian

,District,District_Latitude,District_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Neustadt,53.5519,9.9856,Nasch,53.555528,9.984067,Vegetarian / Vegan Restaurant
1,St.Pauli,53.5569,9.9639,Happenpappen,53.557484,9.970221,Vegetarian / Vegan Restaurant
2,St.Georg,53.5550,10.0122,Loving Hut,53.553915,10.019435,Vegetarian / Vegan Restaurant
3,St.Georg,53.5550,10.0122,Café Koppel,53.558304,10.011780,Vegetarian / Vegan Restaurant
4,Borgfelde,53.5547,10.0344,Loving Hut,53.553915,10.019435,Vegetarian / Vegan Restaurant
5,Ottensen,53.5539,9.9178,Froindlichst,53.556734,9.921648,Vegetarian / Vegan Restaurant


## Show vegetarian/vegan restaurants on a map of Hamburg

In [29]:
# create map of Hamburg using latitude and longitude values
map_hamburg_vegetarian = folium.Map(location=[df_central_distr.loc[0, 'Lat'],df_central_distr.loc[0, 'Long']], zoom_start=13)

# add markers map
for Venue_Latitude, Venue_Longitude, District, Venue in zip(df_central_vegetarian['Venue_Latitude'], df_central_vegetarian['Venue_Longitude'], df_central_vegetarian['District'], df_central_vegetarian['Venue']):
    label = '{}, {}'.format(Venue, District)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        [Venue_Latitude, Venue_Longitude],
        popup=label).add_to(map_hamburg_vegetarian)
   
map_hamburg_vegetarian

## Now explore all central districts according to venue structure and nightlife

In [30]:
print('There are {} unique categories in central districts.'.format(len(central_venues['Venue_Category'].unique())))

There are 193 unique categories in central districts.


In [31]:
# one hot encoding
hamburg_central_onehot = pd.get_dummies(central_venues[['Venue_Category']], prefix="", prefix_sep="")

# add district column back to dataframe
hamburg_central_onehot['District'] = central_venues['District'] 

# move district column to the first column
fixed_columns = [hamburg_central_onehot.columns[-1]] + list(hamburg_central_onehot.columns[:-1])
hamburg_central_onehot = hamburg_central_onehot[fixed_columns]

hamburg_central_onehot.head()

,District,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bistro,Bookstore,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Bridge,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food,Forest,Fountain,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hobby Shop,Hockey Rink,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laundromat,Light Rail Station,Liquor Store,Lounge,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Opera House,Organic Grocery,Outdoor Sculpture,Palatine Restaurant,Paper / Office Supplies Store,Park,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pier,Pizza Place,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spanish Restaurant,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swabian Restaurant,Tanning Salon,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Neustadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Neustadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Neustadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Neustadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [32]:
hamburg_central_onehot.shape

(1164, 194)

In [33]:
hamburg_central_grouped = hamburg_central_onehot.groupby('District').mean().reset_index()
hamburg_central_grouped

,District,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bistro,Bookstore,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Bridge,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food,Forest,Fountain,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hobby Shop,Hockey Rink,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laundromat,Light Rail Station,Liquor Store,Lounge,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Opera House,Organic Grocery,Outdoor Sculpture,Palatine Restaurant,Paper / Office Supplies Store,Park,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pier,Pizza Place,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spanish Restaurant,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swabian Restaurant,Tanning Salon,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Altona-Altstadt,0.00,0.000000,0.00,0.00000,0.00,0.020000,0.000000,0.00,0.000000,0.020000,0.000000,0.040000,0.000000,0.00,0.01,0.00,0.01,0.00,0.000000,0.01,0.00,0.000000,0.000000,0.00,0.01,0.00,0.010000,0.01,0.000000,0.00,0.080000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.030000,0.050000,0.020000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00000,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.01,0.010000,0.000000,0.000000,0.00,0.020000,0.00,0.000000,0.01,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.010000,0.020000,0.01,0.00,0.020000,0.010000,0.01,0.00,0.010000,0.01,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.01,0.00,0.000000,0.00,0.00,0.00,0.01,0.000000,0.000000,0.010000,0.00,0.060000,0.00,0.010000,0.000000,0.00,0.000000,0.020000,0.00,0.01,0.000000,0.000000,0.01,0.01,0.020000,0.010000,0.000000,0.000000,0.000000,0.030000,0.01,0.000000,0.00,0.00,0.070000,0.00,0.01,0.00,0.000000,0.000000,0.00,0.01,0.070000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.00,0.000000,0.000000,0.000000,0.010000,0.010000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.02,0.00,0.00,0.000000,0.000000,0.01,0.010000,0.00,0.01,0.00,0.01,0.01,0.01,0

In [34]:
hamburg_central_grouped.shape

(14, 194)

In [35]:
num_top_venues = 5

for hood in hamburg_central_grouped['District']:
    print("----"+hood+"----")
    temp = hamburg_central_grouped[hamburg_central_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Altona-Altstadt----
                venue  freq
0                Café  0.08
1  Seafood Restaurant  0.07
2          Restaurant  0.07
3           Nightclub  0.06
4        Cocktail Bar  0.05


----Altona-Nord----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.08
2          Bar  0.07
3   Restaurant  0.04
4  Pizza Place  0.03


----Billstedt----
                      venue  freq
0               Supermarket  0.12
1                  Bus Stop  0.09
2            Clothing Store  0.09
3  Mediterranean Restaurant  0.07
4      Fast Food Restaurant  0.07


----Borgfelde----
                venue  freq
0               Hotel  0.14
1                Café  0.10
2  Italian Restaurant  0.04
3             Theater  0.04
4          Restaurant  0.04


----Groß Flottbek----
               venue  freq
0        Supermarket  0.09
1  German Restaurant  0.06
2             Bakery  0.05
3               Park  0.05
4               Café  0.05


----Hamm----
               venue  freq
0               Café

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{}_Most_Common_Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th_Most_Common_Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['District'] = hamburg_central_grouped['District']

for ind in np.arange(hamburg_central_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hamburg_central_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,District,1st_Most_Common_Venue,2nd_Most_Common_Venue,3rd_Most_Common_Venue,4th_Most_Common_Venue,5th_Most_Common_Venue,6th_Most_Common_Venue,7th_Most_Common_Venue,8th_Most_Common_Venue,9th_Most_Common_Venue,10th_Most_Common_Venue
0,Altona-Altstadt,Café,Restaurant,Seafood Restaurant,Nightclub,Cocktail Bar,Bar,Clothing Store,Pub,Hotel,Bakery
1,Altona-Nord,Coffee Shop,Café,Bar,Restaurant,Pizza Place,French Restaurant,Park,Bakery,Theater,Beer Store
2,Billstedt,Supermarket,Bus Stop,Clothing Store,Fast Food Restaurant,Mediterranean Restaurant,Seafood Restaurant,Turkish Restaurant,Café,Hotel,Sandwich Place
3,Borgfelde,Hotel,Café,Theater,Italian Restaurant,Restaurant,Indian Restaurant,German Restaurant,Park,French Restaurant,Ice Cream Shop
4,Groß Flottbek,Supermarket,German Restaurant,Café,Park,Bakery,Bus Stop,Clothing Store,Athletics & Sports,Italian Restaurant,Cosmetics Shop
5,Hamm,Café,Hotel,Supermarket,Bakery,Drugstore,German Restaurant,Greek Restaurant,Park,Chinese Restaurant,Ice Cream Shop
6,Horn,Supermarket,Hotel,Clothing Store,Bakery,Fast Food Restaurant,Café,Mediterranean Restaurant,Seafood Restaurant,German Restaurant,Park
7,Iserbrook,Supermarket,Bakery,Café,Pool,Bus Stop,Park,Ice Cream Shop,Coffee Shop,Indian Restaurant,Steakhouse
8,Lurup,BBQ Joint,Lounge,Bus Stop,German Restaurant,Supermarket,Museum,Shopping Mall,Shipping Store,Restaurant,Plaza
9,Neustadt,Hotel,Café,Coffee Shop,Vietnamese Restaurant,French Restaurant,Seafood Restaurant,Burger Joint,Plaza,Steakhouse,Sushi Restaurant


In [46]:
# set number of clusters
kclusters = 4
hamburg_central_grouped_clustering = hamburg_central_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hamburg_central_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 3, 0, 3, 2, 0, 0, 3], dtype=int32)

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hamburg_merged = df_central_distr

hamburg_merged = hamburg_merged.join(neighborhoods_venues_sorted.set_index('District'), on='District')
hamburg_merged

,Borough,District_No,District,Inhabitants,Area_sqkm,Population_density_per_sqkm,Latitude,Longitude,Lat,Long,Cluster Labels,1st_Most_Common_Venue,2nd_Most_Common_Venue,3rd_Most_Common_Venue,4th_Most_Common_Venue,5th_Most_Common_Venue,6th_Most_Common_Venue,7th_Most_Common_Venue,8th_Most_Common_Venue,9th_Most_Common_Venue,10th_Most_Common_Venue
0,Hamburg-Mitte,105–108,Neustadt,12.920,"2,3",5617,53° 33′ 7″ N,9° 59′ 8″ E,53.5519,9.9856,3,Hotel,Café,Coffee Shop,Vietnamese Restaurant,French Restaurant,Seafood Restaurant,Burger Joint,Plaza,Steakhouse,Sushi Restaurant
1,Hamburg-Mitte,109–112,St.Pauli,22.436,"2,5",8974,53° 33′ 25″ N,9° 57′ 50″ E,53.5569,9.9639,1,Café,Restaurant,Bar,Coffee Shop,Cocktail Bar,Music Venue,Seafood Restaurant,Gastropub,Vietnamese Restaurant,Burger Joint
2,Hamburg-Mitte,113–114,St.Georg,11.384,"2,4",4743,53° 33′ 18″ N,10° 0′ 44″ E,53.5550,10.0122,3,Hotel,Vietnamese Restaurant,Restaurant,Coffee Shop,Burger Joint,Art Gallery,Italian Restaurant,French Restaurant,Café,Theater
3,Hamburg-Mitte,119–120,Borgfelde,7696,"0,8",9620,53° 33′ 17″ N,10° 2′ 4″ E,53.5547,10.0344,3,Hotel,Café,Theater,Italian Restaurant,Restaurant,Indian Restaurant,German Restaurant,Park,French Restaurant,Ice Cream Shop
4,Hamburg-Mitte,121–127,Hamm,38.773,"4,0",9693,53° 33′ 39″ N,10° 3′ 28″ E,53.5608,10.0578,3,Café,Hotel,Supermarket,Bakery,Drugstore,German Restaurant,Greek Restaurant,Park,Chinese Restaurant,Ice Cream Shop
5,Hamburg-Mitte,128–129,Horn,38.799,"5,9",6576,53° 33′ 14″ N,10° 5′ 24″ E,53.5539,10.0900,2,Supermarket,Hotel,Clothing Store,Bakery,Fast Food Restaurant,Café,Mediterranean Restaurant,Seafood Restaurant,German Restaurant,Park
6,Hamburg-Mitte,130,Billstedt,70.355,"17,0",4139,53° 32′ 26″ N,10° 6′ 4″ E,53.5406,10.1011,2,Supermarket,Bus Stop,Clothing Store,Fast Food Restaurant,Mediterranean Restaurant,Seafood Restaurant,Turkish Restaurant,Café,Hotel,Sandwich Place
7,Altona,201–206,Altona-Altstadt,29.305,"2,8",10466,53° 32′ 56″ N,9° 56′ 52″ E,53.5489,9.9478,1,Café,Restaurant,Seafood Restaurant,Nightclub,Cocktail Bar,Bar,Clothing Store,Pub,Hotel,Bakery
8,Altona,207,Sternschanze,8095,"0,5",16190,53° 33′ 42″ N,9° 57′ 44″ E,53.5617,9.9622,1,Bar,Café,Restaurant,Coffee Shop,French Restaurant,Gastropub,Cocktail Bar,Vietnamese Restaurant,Seafood Restaurant,Burger Joint
9,Altona,208–210,Altona-Nord,24.153,"2,2",10979,53° 33′ 47″ N,9° 56′ 43″ E,53.5631,9.9453,1,Coffee Shop,Café,Bar,Restaurant,Pizza Place,French Restaurant,Park,Bakery,Theater,Beer Store


In [48]:
# create map
map_clusters = folium.Map(location=[HHlatitude, HHlongitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hamburg_merged['Lat'], hamburg_merged['Long'], hamburg_merged['District'], hamburg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
# Cluster 1
hamburg_merged.loc[hamburg_merged['Cluster Labels'] == 0, hamburg_merged.columns[[2] + list(range(6, hamburg_merged.shape[1]))]]

,District,Latitude,Longitude,Lat,Long,Cluster Labels,1st_Most_Common_Venue,2nd_Most_Common_Venue,3rd_Most_Common_Venue,4th_Most_Common_Venue,5th_Most_Common_Venue,6th_Most_Common_Venue,7th_Most_Common_Venue,8th_Most_Common_Venue,9th_Most_Common_Venue,10th_Most_Common_Venue
11,Groß Flottbek,53° 33′ 55″ N,9° 52′ 39″ E,53.5653,9.8775,0,Supermarket,German Restaurant,Café,Park,Bakery,Bus Stop,Clothing Store,Athletics & Sports,Italian Restaurant,Cosmetics Shop
12,Lurup,53° 35′ 35″ N,9° 52′ 58″ E,53.5931,9.8828,0,BBQ Joint,Lounge,Bus Stop,German Restaurant,Supermarket,Museum,Shopping Mall,Shipping Store,Restaurant,Plaza
13,Iserbrook,53° 34′ 35″ N,9° 49′ 24″ E,53.5764,9.8233,0,Supermarket,Bakery,Café,Pool,Bus Stop,Park,Ice Cream Shop,Coffee Shop,Indian Restaurant,Steakhouse


In [50]:
# Cluster 2
hamburg_merged.loc[hamburg_merged['Cluster Labels'] == 1, hamburg_merged.columns[[2] + list(range(6, hamburg_merged.shape[1]))]]

,District,Latitude,Longitude,Lat,Long,Cluster Labels,1st_Most_Common_Venue,2nd_Most_Common_Venue,3rd_Most_Common_Venue,4th_Most_Common_Venue,5th_Most_Common_Venue,6th_Most_Common_Venue,7th_Most_Common_Venue,8th_Most_Common_Venue,9th_Most_Common_Venue,10th_Most_Common_Venue
1,St.Pauli,53° 33′ 25″ N,9° 57′ 50″ E,53.5569,9.9639,1,Café,Restaurant,Bar,Coffee Shop,Cocktail Bar,Music Venue,Seafood Restaurant,Gastropub,Vietnamese Restaurant,Burger Joint
7,Altona-Altstadt,53° 32′ 56″ N,9° 56′ 52″ E,53.5489,9.9478,1,Café,Restaurant,Seafood Restaurant,Nightclub,Cocktail Bar,Bar,Clothing Store,Pub,Hotel,Bakery
8,Sternschanze,53° 33′ 42″ N,9° 57′ 44″ E,53.5617,9.9622,1,Bar,Café,Restaurant,Coffee Shop,French Restaurant,Gastropub,Cocktail Bar,Vietnamese Restaurant,Seafood Restaurant,Burger Joint
9,Altona-Nord,53° 33′ 47″ N,9° 56′ 43″ E,53.5631,9.9453,1,Coffee Shop,Café,Bar,Restaurant,Pizza Place,French Restaurant,Park,Bakery,Theater,Beer Store
10,Ottensen,53° 33′ 14″ N,9° 55′ 4″ E,53.5539,9.9178,1,Café,Bakery,Ice Cream Shop,Park,Gym / Fitness Center,Restaurant,German Restaurant,Italian Restaurant,Seafood Restaurant,Pizza Place


In [51]:
# Cluster 3
hamburg_merged.loc[hamburg_merged['Cluster Labels'] == 2, hamburg_merged.columns[[2] + list(range(6, hamburg_merged.shape[1]))]]

,District,Latitude,Longitude,Lat,Long,Cluster Labels,1st_Most_Common_Venue,2nd_Most_Common_Venue,3rd_Most_Common_Venue,4th_Most_Common_Venue,5th_Most_Common_Venue,6th_Most_Common_Venue,7th_Most_Common_Venue,8th_Most_Common_Venue,9th_Most_Common_Venue,10th_Most_Common_Venue
5,Horn,53° 33′ 14″ N,10° 5′ 24″ E,53.5539,10.0900,2,Supermarket,Hotel,Clothing Store,Bakery,Fast Food Restaurant,Café,Mediterranean Restaurant,Seafood Restaurant,German Restaurant,Park
6,Billstedt,53° 32′ 26″ N,10° 6′ 4″ E,53.5406,10.1011,2,Supermarket,Bus Stop,Clothing Store,Fast Food Restaurant,Mediterranean Restaurant,Seafood Restaurant,Turkish Restaurant,Café,Hotel,Sandwich Place


In [52]:
# Cluster 4
hamburg_merged.loc[hamburg_merged['Cluster Labels'] == 3, hamburg_merged.columns[[2] + list(range(6, hamburg_merged.shape[1]))]]

,District,Latitude,Longitude,Lat,Long,Cluster Labels,1st_Most_Common_Venue,2nd_Most_Common_Venue,3rd_Most_Common_Venue,4th_Most_Common_Venue,5th_Most_Common_Venue,6th_Most_Common_Venue,7th_Most_Common_Venue,8th_Most_Common_Venue,9th_Most_Common_Venue,10th_Most_Common_Venue
0,Neustadt,53° 33′ 7″ N,9° 59′ 8″ E,53.5519,9.9856,3,Hotel,Café,Coffee Shop,Vietnamese Restaurant,French Restaurant,Seafood Restaurant,Burger Joint,Plaza,Steakhouse,Sushi Restaurant
2,St.Georg,53° 33′ 18″ N,10° 0′ 44″ E,53.5550,10.0122,3,Hotel,Vietnamese Restaurant,Restaurant,Coffee Shop,Burger Joint,Art Gallery,Italian Restaurant,French Restaurant,Café,Theater
3,Borgfelde,53° 33′ 17″ N,10° 2′ 4″ E,53.5547,10.0344,3,Hotel,Café,Theater,Italian Restaurant,Restaurant,Indian Restaurant,German Restaurant,Park,French Restaurant,Ice Cream Shop
4,Hamm,53° 33′ 39″ N,10° 3′ 28″ E,53.5608,10.0578,3,Café,Hotel,Supermarket,Bakery,Drugstore,German Restaurant,Greek Restaurant,Park,Chinese Restaurant,Ice Cream Shop


In [109]:
venue_kind = ['Bar', 'Cocktail Bar', 'Nightclub', 'Pub']
cond1 = hamburg_merged["1st_Most_Common_Venue"].isin(venue_kind)    
cond2 = hamburg_merged["2nd_Most_Common_Venue"].isin(venue_kind) 
cond3 = hamburg_merged["3rd_Most_Common_Venue"].isin(venue_kind)
cond4 = hamburg_merged["4th_Most_Common_Venue"].isin(venue_kind) 
cond5 = hamburg_merged["5th_Most_Common_Venue"].isin(venue_kind) 
cond6 = hamburg_merged["6th_Most_Common_Venue"].isin(venue_kind) 

hh_nightlife_districts = hamburg_merged.loc[cond1 | cond2 | cond3 | cond4 | cond5]

hh_nightlife_districts

,Borough,District_No,District,Inhabitants,Area_sqkm,Population_density_per_sqkm,Latitude,Longitude,Lat,Long,Cluster Labels,1st_Most_Common_Venue,2nd_Most_Common_Venue,3rd_Most_Common_Venue,4th_Most_Common_Venue,5th_Most_Common_Venue,6th_Most_Common_Venue,7th_Most_Common_Venue,8th_Most_Common_Venue,9th_Most_Common_Venue,10th_Most_Common_Venue
1,Hamburg-Mitte,109–112,St.Pauli,22.436,"2,5",8974,53° 33′ 25″ N,9° 57′ 50″ E,53.5569,9.9639,1,Café,Restaurant,Bar,Coffee Shop,Cocktail Bar,Music Venue,Seafood Restaurant,Gastropub,Vietnamese Restaurant,Burger Joint
7,Altona,201–206,Altona-Altstadt,29.305,"2,8",10466,53° 32′ 56″ N,9° 56′ 52″ E,53.5489,9.9478,1,Café,Restaurant,Seafood Restaurant,Nightclub,Cocktail Bar,Bar,Clothing Store,Pub,Hotel,Bakery
8,Altona,207,Sternschanze,8095,"0,5",16190,53° 33′ 42″ N,9° 57′ 44″ E,53.5617,9.9622,1,Bar,Café,Restaurant,Coffee Shop,French Restaurant,Gastropub,Cocktail Bar,Vietnamese Restaurant,Seafood Restaurant,Burger Joint
9,Altona,208–210,Altona-Nord,24.153,"2,2",10979,53° 33′ 47″ N,9° 56′ 43″ E,53.5631,9.9453,1,Coffee Shop,Café,Bar,Restaurant,Pizza Place,French Restaurant,Park,Bakery,Theater,Beer Store
